In [ ]:
from psychopy import data, visual, core, event, monitors # import some libraries from PsychoPy
import numpy as np
import psychopy.tools.monitorunittools as unittools
import pandas as pd
import time

In [ ]:
# Stim Settings
ntrials    = 4
nUni       = 4 #int(ntrials/2)
nCause     = 0 #ntrials-nUni
Ns         = [3,4,5,6]
sigma_s    = 4
sigma      = 4
rangemax    = 100


# Subject Settings
subID = 'Jenn'
blockID = 1

# Seed stimuli
N       = np.tile(Ns, int(ntrials/len(Ns)))
feeder  = np.array([1] * nCause + [0] * nUni)
mu      = np.random.normal(int(rangemax/2),sigma_s,nCause).tolist() + [None]*nUni

# Get stimuli x-values
x = list()
for i_trial in range(ntrials):
    if feeder[i_trial]==1:
        x.append(np.random.normal(mu[i_trial],sigma,[1,N[i_trial]]))
    elif feeder[i_trial]==0:
        x.append(np.random.uniform(0,100,[1,N[i_trial]]))

# Randomize vectors together
randomize = np.arange(ntrials)
np.random.shuffle(randomize)

N      = N[randomize]
feeder = feeder[randomize]
mu     = [mu[i] for i in randomize]
x      = [x[i] for i in randomize]

# Monitor settings
mon = config_monitor()

# Screen settings
screenheight = 900
screenwidth = 1440

# Scale settings
scalewidth = 800

sloth = 150
slotw = 150

bg_colour = [0,0,0]
win = visual.Window(
    size=[screenwidth, screenheight],
    units="pix",
    fullscr=False,
    color=bg_colour,
    colorSpace='rgb'
)

# Clock settings
timer = core.Clock()

# Line settings
line = visual.Line(
    win=win,
    units="pix",
    lineColor=[-1, -1, -1]
)

# Create dictionary
data = {'Trial_ID':[],
        'Resp_loc':[], 
        'Resp_conf':[], 
        'RT_resp':[], 
        'RT_conf':[],
        'Wager':[],
        'x':[],
        'N':[],
        'x_mean':[]}

# Create dataframe
df = pd.DataFrame(data)

for trial in range(ntrials):    
    # Mouse settings + initialization
    #xspawn  = np.random.uniform(-screenwidth/2,screenwidth/2)
    myMouse = event.Mouse(visible = True, win = win)#,newPos = [xspawn,0]) to spawn cursor at random x-coord

    #rect_locs, rect_width, RTs = get_rect_response(np.around(x[trial][0]), rect_locs, rect_widths, RTs)
    resp_loc, rt_resp = get_point_response(x[trial][0])

    resp_conf, rt_conf, wager = get_conf_response(x[trial][0],resp_loc)
    
    currtrial = pd.DataFrame({'Trial_ID':[trial+1],
                              'Resp_loc':[resp_loc],
                              'Resp_conf':[resp_conf],
                              'RT_resp':[rt_resp],
                              'RT_conf':[rt_conf],
                              'Wager':[wager],
                              'x': [x[trial]],
                              'N': [N[trial]],
                              'x_mean': [np.mean(x[trial])]})
    df = df.append(currtrial,ignore_index=True,sort=False)

timestr = time.strftime("%Y%m%d-%H%M%S")
filename = str('Numbers-S' + subID + '-B' + str(blockID) + '-' + timestr + '.csv')
print(filename)
df.to_csv(filename)
           
win.close()
core.quit()

In [ ]:
df

timestr = time.strftime("%Y%m%d-%H%M%S")
filename = str('Numbers-S' + subID + '-B' + str(blockID) + '-' + timestr + '.csv')
print(filename)
df.to_csv(filename)
           
win.close()
core.quit()

In [ ]:
def config_monitor():
    my_monitor = monitors.Monitor(name='mac_monitor')
    my_monitor.setSizePix((1280, 800))
    my_monitor.setWidth(20)
    my_monitor.setDistance(100)
    my_monitor.saveMon()
    return my_monitor

In [ ]:
def config_rect_cursor(xpos, width):
    
    rect = visual.Rect(
        win=win,
        units="pix",
        width=width,
        height=30,
        fillColor=[1, -1, -1],
        opacity=0.3,
        pos = [x2pix(xpos),0]
    )
    return rect


In [ ]:
def get_point_response(x_):
    mouse_down_detected = False
    timer.reset()

    while not mouse_down_detected:
        mouse_loc_pix = myMouse.getPos()
        xpos = pix2x(mouse_loc_pix[0])
        mouse_click = myMouse.getPressed()
        
        if xpos<0:
            xpos = 0
        elif xpos>100:
            xpos = 100
            
        plot_numstim(x_)
        disp_numberline()   
        
        if mouse_click[0]: # button 0 is pressed
            if not mouse_down_detected: # initial detection
                rt = timer.getTime()
                resp_loc = xpos
                mouse_down_detected = True
                        
        else: # button 0 is NOT pressed
            mouse_down_detected = False
            line = config_line(xpos,(1,0,0))
            line.draw()
            draw_dispnum(xpos,(x2pix(xpos)-12,-170),20)
       
            win.flip()
        
    return resp_loc, rt

In [ ]:
def get_conf_response(x_,resp_loc):
    space_down_detected = False
    timer.reset()
    
    while not space_down_detected:
        mouse_loc_pix = myMouse.getPos()
        resp_conf_ = abs(pix2x(mouse_loc_pix[0])-resp_loc)*2
        pts = 100*np.exp(-resp_conf_/10)
        
        plot_numstim(x_)
        disp_numberline()
        
        keys = event.getKeys(['space'])
        if keys != []:
            if keys[0] == 'space': # space bar is pressed
                if not space_down_detected: # initial detection
                    rt        = timer.getTime()
                    resp_conf = resp_conf_
                    wager     = round(int(pts))
                    space_down_detected = True
                rect=config_rect_cursor(resp_loc,resp_conf_*scalewidth/rangemax)
                rect.draw()
                
                # plot feedback
                line = config_line(np.mean(x_),[0, 1, 0])
                line.draw()

                correct = (resp_loc - resp_conf/2) < np.mean(x_) < (resp_loc + resp_conf/2)

#                 print(rect_loc)
#                 print(rect_width)
#                 print(np.mean(x_))

                if correct:
                    fb2 = '+$' + str(wager)
                    fbcolor = (-1,1,-1)
                else:
                    wager = 0
                    fb2 = '+$0'
                    fbcolor = (1,-1,-1)

                #draw_fb(fb1, (0,0,1), (x2pix(np.mean(x_))-10,60))
                draw_fb(fb2, fbcolor, (-10,-300),40)
                disp_numberline()
                win.flip() 
                core.wait(1)

        else: # space bar is NOT pressed
            space_down_detected = False
            line = config_line(resp_loc,(1,0,0))
            line.draw()
            draw_dispnum(resp_loc,(x2pix(resp_loc)-12,-170),20)
            
            rect=config_rect_cursor(resp_loc,resp_conf_*scalewidth/rangemax)
            rect.draw()
            pt_text = ' $'+ str(round(int(pts)))
            draw_fb(pt_text,(-1,1,-1),(-10,-300),40)
            
            win.flip()

    return resp_conf, rt, wager

In [ ]:
def config_line(xpos,colour):
    line.start = [x2pix(xpos), -150]
    line.end = [x2pix(xpos), +150]
    line.setColor(colour,'rgb')
    #line.lineColorSpace='rgb'
    return line

In [ ]:
def draw_fb(text, color, pos, height):
    text = visual.TextStim(win=win, text=text, color=color, colorSpace='rgb',units='pix', alignHoriz='left', alignVert='center', pos=pos, height=height)
    text.draw()

In [ ]:
def draw_dispnum(dispnum,posi,height):
    num = visual.TextStim(
                    win=win,
                    wrapWidth=350,
                    text=str(int(np.round(dispnum))),
                    pos=posi,
                    alignHoriz='left',
                    units="pix",
                    height = height,
                    color = (1,1,1)
                )
    num.draw()
    #return num

In [ ]:
def plot_numstim(x_):
    margh = (screenheight - sloth*5)/2
    margw = (screenwidth - (len(x_)-1)*slotw - len(x_)*7) /2
    
    ipos = 1
    
    for number in x_:
        num = visual.TextStim(
            win=win,
            wrapWidth=350,
            alignHoriz = 'left',
            height=30,
            color = (-1,-1,-1)
        )
        num.text = str(int(number))

        row = np.floor((ipos-1)/6) + 1
        col = ipos-1#-(row-1)*6

        ypos = -((margh + row*sloth)-screenheight/2)
        xpos = (margw + col*slotw)-screenwidth/2

        num.pos = (xpos,ypos)

        num.draw()

        ipos = ipos + 1

In [ ]:
def x2pix(x):
    pix = x*scalewidth/rangemax - scalewidth/2
    return pix

In [ ]:
def pix2x(pix):
    x = (pix+scalewidth/2)*rangemax/scalewidth
    return x

In [ ]:
def disp_numberline():
    ticks = [0,25,50,75,100]
    for ii in ticks:
        line.start = [x2pix(ii), -30]
        line.end = [(x2pix(ii)), +30]
        line.setColor((-1, -1, 1),'rgb')
        line.draw()
        
        draw_dispnum(ii,(x2pix(ii)-10,+30),12)
    
    #horizontal line
    line.start = [x2pix(0),0]
    line.end = [x2pix(100),0]
    line.setColor((-1, -1, 1),'rgb')
    line.draw()
    

In [ ]:
def config_cross_cursor(xpos):
# Draw cross cursor
    size = 0.1
    cross = visual.ShapeStim(win, 
        vertices=((xpos, -size), (xpos, size), (xpos,0), (xpos-size,0), (xpos+size, 0)),
        lineWidth=4,
        closeShape=False,
        lineColor="red"
    )
    return cross

In [ ]:
test =1

if (test & ~5%2) | ~4%2:
    print('yes')